# Raspador de dados do judiciário paulista
Código para raspar dados do E-SAJ do Tribunal de Justiça do Estado de São Paulo (TJ-SP). 

*Matheus Arruda

# Exemplos de uso
### Quais deputados se autobeneficiaram ao votar a favor da lei de improbidade administrativa
O código vai buscar informações, via API da Câmara dos Deputados, da votação do PL n° 2505, de 2021 - sob ID DE VOTAÇÃO: 2184458-184. <br>
Na 2ª etapa, iremos automatizar searchs no e-SAJ do TJ-SP e raspar as informações. 

# Preparando o ambiente
Para a raspagem dos dados, utilizaremos as bibliotecas Selenium. Para a análise, utilizaremos o Pandas. 
Já para a representação dos dados utilizaremos o matplotlib e Seaborn. 

In [1]:

#Biblioteca para raspagem 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By

#Bibliotecas para várias funções relacionadas à tempo
import time
from datetime import datetime

#Bibliotecas para análise dos dados
import pandas as pd 
import numpy as np 
import requests as requests
import json
import matplotlib.pyplot as plt 
import seaborn as sns


In [2]:
#Linhas para configurar a IDE para exibir o máximo de colunas e largura máxima de colunas
pd.set_option('display.max_columns', None)

# Conexão API Câmara dos Deputados

In [3]:
#Vamos criar nosso cabeçalho para aceitar as requisiões em json
headers = {
    "Accept": "Application/json"
}

### Identificar os id's da PL

In [4]:
votos_pl = requests.get('https://dadosabertos.camara.leg.br/api/v2/proposicoes/2184458/votacoes?ordem=DESC&ordenarPor=dataHoraRegistro', headers=headers)
votos_pl = votos_pl.json()

votos_pl = pd.json_normalize(votos_pl['dados'])
votos_pl.head(6)


,id,uri,data,dataHoraRegistro,siglaOrgao,uriOrgao,uriEvento,proposicaoObjeto,uriProposicaoObjeto,descricao,aprovacao
0,2184458-186,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-06,2021-10-06T16:58:25,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Aprovada a Redação Final assinada pelo Relator...,1.0
1,2184458-184,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-06,2021-10-06T16:57:25,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Aprovada a Emenda do Senado Federal nº 8. Sim:...,1.0
2,2184458-172,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-05,2021-10-05T20:05:40,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Rejeitada a Emenda do Senado Federal nº 4 ao P...,0.0
3,2184458-169,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-05,2021-10-05T19:47:45,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,"Aprovadas as Emendas do Senado Federal nºs 1, ...",1.0
4,2184458-161,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-05,2021-10-05T18:34:35,PLEN,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,None,None,Rejeitado o Requerimento. Sim: 60; não: 346; t...,0.0
5,2301348-3,https://dadosabertos.camara.leg.br/api/v2/vota...,2021-10-05,2021-10-05T13:23:52,CPD,https://dadosabertos.camara.leg.br/api/v2/orga...,https://dadosabertos.camara.leg.br/api/v2/even...,REQ 76/2021 CPD,https://dadosabertos.camara.leg.br/api/v2/prop...,Aprovado.,1.0


### Lista votos

In [5]:
lista_votos = requests.get('https://dadosabertos.camara.leg.br/api/v2/votacoes/2184458-184/votos', headers=headers)
lista_votos = lista_votos.json()

lista_votos = pd.json_normalize(lista_votos['dados'])
lista_votos

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email
0,Sim,2021-10-06T16:55:06,74352,https://dadosabertos.camara.leg.br/api/v2/depu...,Nilson Pinto,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,PA,56,https://www.camara.leg.br/internet/deputado/ba...,None
1,Sim,2021-10-06T16:54:47,74366,https://dadosabertos.camara.leg.br/api/v2/depu...,João Campos,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,GO,56,https://www.camara.leg.br/internet/deputado/ba...,None
2,Sim,2021-10-06T16:54:10,204553,https://dadosabertos.camara.leg.br/api/v2/depu...,Pastor Sargento Isidório,AVANTE,https://dadosabertos.camara.leg.br/api/v2/part...,BA,56,https://www.camara.leg.br/internet/deputado/ba...,dep.pastorsargentoisidorio@camara.leg.br
3,Não,2021-10-06T16:54:02,141553,https://dadosabertos.camara.leg.br/api/v2/depu...,Vanderlei Macris,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,None
4,Não,2021-10-06T16:53:57,204363,https://dadosabertos.camara.leg.br/api/v2/depu...,Osires Damaso,PSC,https://dadosabertos.camara.leg.br/api/v2/part...,TO,56,https://www.camara.leg.br/internet/deputado/ba...,None
...,...,...,...,...,...,...,...,...,...,...,...
418,Não,2021-10-06T16:45:03,204548,https://dadosabertos.camara.leg.br/api/v2/depu...,Heitor Freire,PSL,https://dadosabertos.camara.leg.br/api/v2/part...,CE,56,https://www.camara.leg.br/internet/deputado/ba...,None
419,Sim,2021-10-06T16:45:01,76874,https://dadosabertos.camara.leg.br/api/v2/depu...,Marcelo Freixo,PSB,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,56,https://www.camara.leg.br/internet/deputado/ba...,None
420,Sim,2021-10-06T16:45:01,160556,https://dadosabertos.camara.leg.br/api/v2/depu...,Padre João,PT,https://dadosabertos.camara.leg.br/api/v2/part...,MG,56,https://www.camara.leg.br/internet/deputado/ba...,dep.padrejoao@camara.leg.br
421,Sim,2021-10-06T16:44:59,73801,https://dadosabertos.camara.leg.br/api/v2/depu...,Renildo Calheiros,PCdoB,https://dadosabertos.camara.leg.br/api/v2/part...,PE,56,https://www.camara.leg.br/internet/deputado/ba...,dep.renildocalheiros@camara.leg.br


### Informações adicionais dos deputados

In [6]:
nome_deputado = []

for i, row in lista_votos.iterrows():
    deputado_id = row['deputado_.id']
    deputados_infos = requests.get(f'https://dadosabertos.camara.leg.br/api/v2/deputados/{deputado_id}')
    deputados_infos = deputados_infos.json()
    nome_deputado.append(deputados_infos['dados']['nomeCivil'])
    print(deputado_id)

lista_votos['NomeCivilDeputado'] = nome_deputado

74352
74366
204553
141553
204363
160758
204505
178934
178884
217480
204547
204414
178896
178927
204364
204387
204531
160592
141516
204473
178996
204482
160591
204395
141513
141458
178986
141434
157130
204559
160976
196358
178937
178956
204527
74090
204398
204428
141459
74317
204377
204515
178933
73692
204539
160610
198783
204392
74376
166402
160553
179001
74537
74253
160527
217330
74439
178910
74646
204571
73486
74052
209189
204449
204431
178964
178921
204488
66179
90201
204386
74159
197438
178970
204534
164359
160601
178985
178912
74158
204511
204360
204408
204475
73466
74419
178873
160528
204502
141405
204467
178843
204518
178871
73788
160510
133439
160508
177282
204532
73604
204427
178909
204522
204446
141391
204410
160604
191945
204479
204405
178835
195866
178948
204369
215042
204413
160666
204481
160642
204366
204537
204485
74749
141523
141408
143084
160665
204407
75431
204497
204538
205548
115746
204566
92776
178939
204418
178832
204393
204498
74467
194260
74254
204372
136811
204

In [7]:
#Vamos criar um DataFrame contendo apenas os deputados de SP
bd_deputados_sp = lista_votos.query('`deputado_.siglaUf` == "SP" and tipoVoto == "Sim"')
bd_deputados_sp

,tipoVoto,dataRegistroVoto,deputado_.id,deputado_.uri,deputado_.nome,deputado_.siglaPartido,deputado_.uriPartido,deputado_.siglaUf,deputado_.idLegislatura,deputado_.urlFoto,deputado_.email,NomeCivilDeputado
26,Sim,2021-10-06T16:51:53,178986,https://dadosabertos.camara.leg.br/api/v2/depu...,Nilto Tatto,PT,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.niltotatto@camara.leg.br,NILTO IGNACIO TATTO
30,Sim,2021-10-06T16:51:37,160976,https://dadosabertos.camara.leg.br/api/v2/depu...,Tiririca,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.tiririca@camara.leg.br,FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA
34,Sim,2021-10-06T16:51:12,204527,https://dadosabertos.camara.leg.br/api/v2/depu...,Geninho Zuliani,DEM,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,None,EUGENIO JOSÉ ZULIANI
76,Sim,2021-10-06T16:49:06,160601,https://dadosabertos.camara.leg.br/api/v2/depu...,Pr. Marco Feliciano,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.pr.marcofeliciano@camara.leg.br,MARCO ANTÔNIO FELICIANO
80,Sim,2021-10-06T16:49:00,204511,https://dadosabertos.camara.leg.br/api/v2/depu...,David Soares,DEM,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.davidsoares@camara.leg.br,DAVID BEZERRA RIBEIRO SOARES
100,Sim,2021-10-06T16:48:23,73604,https://dadosabertos.camara.leg.br/api/v2/depu...,Rui Falcão,PT,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.ruifalcao@camara.leg.br,RUI GOETHE DA COSTA FALCÃO
121,Sim,2021-10-06T16:47:41,204537,https://dadosabertos.camara.leg.br/api/v2/depu...,Enrico Misasi,PV,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,None,ENRICO VAN BLARCUM DE GRAAFF MISASI
122,Sim,2021-10-06T16:47:40,204485,https://dadosabertos.camara.leg.br/api/v2/depu...,Luiz Carlos Motta,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.luizcarlosmotta@camara.leg.br,LUIZ CARLOS MOTTA
147,Sim,2021-10-06T16:47:18,217036,https://dadosabertos.camara.leg.br/api/v2/depu...,Henrique do Paraíso,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,None,HENRIQUE STEIN SCIASCIO
163,Sim,2021-10-06T16:47:06,73441,https://dadosabertos.camara.leg.br/api/v2/depu...,Celso Russomanno,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,SP,56,https://www.camara.leg.br/internet/deputado/ba...,dep.celsorussomanno@camara.leg.br,CELSO UBIRAJARA RUSSOMANNO


# Ex. 1 - Raspagem dos dados por NOME DA PARTE

In [8]:
driver = webdriver.Chrome()
driver.get('https://esaj.tjsp.jus.br/cpopg/open.do')
driver.maximize_window()

# --- LISTAS PARA ARMAZENAR DADOS ---
nome_deputados_lista = []
partidos_lista = []
num_processos_lista = []
classeprocesso_lista = []
assuntoprocesso_lista = []

def realizar_pesquisa(driver, nome_deputado):
    """Realiza a pesquisa por nome completo no site."""
    try:
        botao_tipo_pesquisa = driver.find_element(By.ID, "cbPesquisa")
        select_tipo_pesquisa = Select(botao_tipo_pesquisa)
        select_tipo_pesquisa.select_by_index(1)  # Seleciona para pesquisar por NOME

        campo_nome = driver.find_element(By.ID, "campo_NMPARTE")
        campo_nome.clear()
        campo_nome.send_keys(nome_deputado)
        
        driver.implicitly_wait(15)
        
        driver.find_element(By.ID, "pesquisarPorNomeCompleto").click() #Seleciona a caixa para pesquisar por nome completo
        
        driver.implicitly_wait(10)

        driver.find_element(By.ID, "botaoConsultarProcessos").click() #Clica no botão para pesquisar
        
    except NoSuchElementException:
        print(f"Erro ao realizar pesquisa para {nome_deputado}")
        return False  # Indica falha na pesquisa
    return True # Indica sucesso na pesquisa


def extrair_dados_da_tabela(driver, nome_deputado, partido_deputado):
    """Extrai dados da tabela de processos, se presente."""
    nome_lista_temp = []
    partidos_lista_temp = []
    num_processos_lista_temp = []
    classeprocesso_lista_temp = []
    assuntoprocesso_lista_temp = []

    try:
        lista_processo_elementos = driver.find_elements(By.CLASS_NAME, "linkProcesso")
        classe_processo_elementos = driver.find_elements(By.CLASS_NAME, "classeProcesso")
        assunto_processo_elementos = driver.find_elements(By.CLASS_NAME, "assuntoPrincipalProcesso")

        if not lista_processo_elementos: # Verifica se encontrou processos
            print(f"Nenhum processo encontrado na tabela para {nome_deputado}")
            return [], [], [], [], []

        for i in range(len(lista_processo_elementos)):
            nome_lista_temp.append(nome_deputado)
            partidos_lista_temp.append(partido_deputado)
            num_processos_lista_temp.append(lista_processo_elementos[i].text)
            classeprocesso_lista_temp.append(classe_processo_elementos[i].text)
            assuntoprocesso_lista_temp.append(assunto_processo_elementos[i].text)

    except NoSuchElementException:
        print(f"Erro ao extrair dados da tabela para {nome_deputado}")
        return [], [], [], [], [] # Retorna listas vazias em caso de erro

    return nome_lista_temp, partidos_lista_temp, num_processos_lista_temp, classeprocesso_lista_temp, assuntoprocesso_lista_temp


def extrair_detalhes_processo(driver, nome_deputado, partido_deputado):
    """Extrai detalhes do processo se estiver na página 'show.do?'."""
    try:
        nprocesso = driver.find_element(By.ID, "numeroProcesso").text
        processo_clase = driver.find_element(By.ID, "labelClasseProcesso").text
        processo_assunto = driver.find_element(By.ID, "assuntoProcesso").text

        nome_deputados_lista.append(nome_deputado)
        partidos_lista.append(partido_deputado)
        num_processos_lista.append(nprocesso)
        classeprocesso_lista.append(processo_clase)
        assuntoprocesso_lista.append(processo_assunto)

    except NoSuchElementException:
        print(f"Erro ao extrair detalhes do processo para {nome_deputado}")


def voltar_para_pesquisa(driver):
    """Limpa o campo de pesquisa para a próxima busca."""
    try:
        campo_nome = driver.find_element(By.ID, "campo_NMPARTE")
        campo_nome.clear()

        time.sleep(10)
        
    except NoSuchElementException:
        print("Erro ao limpar campo de pesquisa.")


# --- LOOP PRINCIPAL ---
for i, row in bd_deputados_sp.iterrows():  
    nome_deputado = row['NomeCivilDeputado']
    partido_deputado = row['deputado_.siglaPartido']

    print(f"Pesquisando por: {nome_deputado} ({partido_deputado})")

    if realizar_pesquisa(driver, nome_deputado): # Realiza pesquisa e verifica sucesso
        url = driver.current_url
        if "show.do?" in str(url):
            extrair_detalhes_processo(driver, nome_deputado, partido_deputado)
            try:
                driver.find_element(By.CLASS_NAME, 'unj-link-back').click() # Voltar para lista
            except NoSuchElementException:
                print("Erro ao clicar em 'Voltar' para lista de processos.")

        else: # Página de lista de processos (ou "nenhum processo encontrado")
            n, pa, np, cp, ap = extrair_dados_da_tabela(driver, nome_deputado, partido_deputado)
            nome_deputados_lista.extend(n)
            partidos_lista.extend(pa)
            num_processos_lista.extend(np)
            classeprocesso_lista.extend(cp)
            assuntoprocesso_lista.extend(ap)

        voltar_para_pesquisa(driver) # Limpa campo para próxima pesquisa
    else:
        print(f"Falha na pesquisa inicial para {nome_deputado}. Pulando para o próximo deputado.")

driver.quit()

Pesquisando por: NILTO IGNACIO TATTO (PT)
Nenhum processo encontrado na tabela para NILTO IGNACIO TATTO
Pesquisando por: FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA (PL)
Nenhum processo encontrado na tabela para FRANCISCO EVERARDO TIRIRICA OLIVEIRA SILVA
Pesquisando por: EUGENIO JOSÉ ZULIANI (DEM)
Pesquisando por: MARCO ANTÔNIO FELICIANO (PL)
Pesquisando por: DAVID BEZERRA RIBEIRO SOARES (DEM)
Pesquisando por: RUI GOETHE DA COSTA FALCÃO (PT)
Nenhum processo encontrado na tabela para RUI GOETHE DA COSTA FALCÃO
Pesquisando por: ENRICO VAN BLARCUM DE GRAAFF MISASI (PV)
Pesquisando por: LUIZ CARLOS MOTTA (PL)
Pesquisando por: HENRIQUE STEIN SCIASCIO (REPUBLICANOS)
Pesquisando por: CELSO UBIRAJARA RUSSOMANNO (REPUBLICANOS)
Pesquisando por: LUIZ PAULO TEIXEIRA FERREIRA (PT)
Pesquisando por: VINICIUS RAPOZO DE CARVALHO (REPUBLICANOS)
Nenhum processo encontrado na tabela para VINICIUS RAPOZO DE CARVALHO
Pesquisando por: PAULO ROBERTO FREIRE DA COSTA (PL)
Pesquisando por: BENEDITO ROBERTO ALVES 

### Criação do DataFrame

In [9]:
data = {'Nome':nome_deputados_lista, 'Partido':partidos_lista, 'Processo':num_processos_lista, 'Classe':classeprocesso_lista, 'Assunto':assuntoprocesso_lista}

bd_deputados_sp = pd.DataFrame(data)

bd_deputados_sp

,Nome,Partido,Processo,Classe,Assunto
0,EUGENIO JOSÉ ZULIANI,DEM,1037674-83.2023.8.26.0100,Procedimento Comum Cível,DIREITO PROCESSUAL CIVIL E DO TRABALHO
1,EUGENIO JOSÉ ZULIANI,DEM,1000297-15.2025.8.26.0066,Ação Popular,Atos Administrativos
2,EUGENIO JOSÉ ZULIANI,DEM,1000426-85.2025.8.26.0400,Mandado de Segurança Cível,Concurso Público / Edital
3,EUGENIO JOSÉ ZULIANI,DEM,1001882-07.2024.8.26.0400,Procedimento do Juizado Especial Cível,Indenização por Dano Moral
4,EUGENIO JOSÉ ZULIANI,DEM,1001632-71.2024.8.26.0400,"Crimes de Calúnia, Injúria e Difamação de Comp...",Simples
...,...,...,...,...,...
186,LUIZA ERUNDINA DE SOUSA,PSOL,0728959-30.1990.8.26.0053,Ação Popular,
187,LUIZA ERUNDINA DE SOUSA,PSOL,0707367-61.1989.8.26.0053,Ação Popular,Atos Administrativos
188,CARLOS ALBERTO ROLIM ZARATTINI,PT,0007357-67.2003.8.26.0053,Ação Popular,Utilização de bens públicos
189,CARLOS ALBERTO ROLIM ZARATTINI,PT,0418437-07.1996.8.26.0053,Cumprimento de sentença,Improbidade Administrativa


### Análise do DataFrame

In [10]:
bd_deputados_sp['Assunto'].value_counts()

Assunto
Indenização por Dano Moral                                   26
                                                             17
Improbidade Administrativa                                   14
IPTU/ Imposto Predial e Territorial Urbano                   11
Dano ao Erário                                                7
                                                             ..
Sucessões                                                     1
Crimes contra a Fé Pública                                    1
Inexequibilidade do Título / Inexigibilidade da Obrigação     1
Citação                                                       1
Utilização de bens públicos                                   1
Name: count, Length: 72, dtype: int64

In [11]:
bd_deputados_sp['Assunto'].value_counts().nlargest(50)

Assunto
Indenização por Dano Moral                                                                                   26
                                                                                                             17
Improbidade Administrativa                                                                                   14
IPTU/ Imposto Predial e Territorial Urbano                                                                   11
Dano ao Erário                                                                                                7
Violação aos Princípios Administrativos                                                                       6
Roubo                                                                                                         5
Atos Administrativos                                                                                          5
Usucapião Extraordinária                                                                        

# Ex.2: Buscas por número do processo
Neste segundo exemplo utilizaremos dados obtidos via API do Conselho Nacional de Justiça (CNJ). 
O Datajud, API do CNJ, transfere dados via ElasticSearch. Segundo a própria documentação, "é a base nacional de metadados processuais do poder judiciário brasileiro. Sua finalidade é centralizar e padronizar as informações processuais de todo o país com o objetivo subsidiar o Sistema de Estatística do Poder Judiciário – SIESPJ conforme estabelecido pela Resolução CNJ n. 331/2020. A partir do Datajud é possível a geração de estatísticas e análises sobre a atividade judiciária em todo o território nacional."

Leia aqui os termos de uso da API: https://datajud-wiki.cnj.jus.br/api-publica/termo-uso

In [24]:
# URL da API
url = "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search"

#Substituir <API Key> pela Chave Pública
headers = {
  'Authorization': 'APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
  'Content-Type': 'application/json'
}

In [25]:
payload = json.dumps({
    "size":10000,
    "query": {
            "match":{"classe.codigo": 64} ##Ações de improbidade administrativa
            },
    "sort": [{"dataAjuizamento": {"order":"asc"}}]
    }
)

In [26]:
response = requests.request("POST", url, headers=headers, data=payload)
processo_json = response.json()

In [27]:
processo_json['hits'].keys()

dict_keys(['total', 'max_score', 'hits'])

In [28]:
processo_json['hits']['hits'][0]

{'_index': 'api_publica_tjsp',
 '_id': 'TJSP_G1_20001169619968260103',
 '_score': None,
 '_source': {'numeroProcesso': '20001169619968260103',
  'classe': {'codigo': 64, 'nome': 'Ação Civil de Improbidade Administrativa'},
  'sistema': {'codigo': 3, 'nome': 'SAJ'},
  'formato': {'codigo': 2, 'nome': 'Físico'},
  'tribunal': 'TJSP',
  'dataHoraUltimaAtualizacao': '2024-12-12T23:53:18.355Z',
  'grau': 'G1',
  '@timestamp': '2025-02-06T10:03:06.016842036Z',
  'dataAjuizamento': '1996-05-23T08:00:00.000Z',
  'movimentos': [{'complementosTabelados': [{'codigo': 2,
      'valor': 2,
      'nome': 'sorteio',
      'descricao': 'tipo_de_distribuicao_redistribuicao'}],
    'codigo': 26,
    'nome': 'Distribuição',
    'dataHora': '1996-05-23T08:00:00.000Z'},
   {'complementosTabelados': [{'codigo': 18,
      'valor': 40,
      'nome': 'outros motivos',
      'descricao': 'motivo_da_remessa'}],
    'codigo': 982,
    'nome': 'Remessa',
    'dataHora': '2024-08-21T14:03:51.000Z'},
   {'codigo': 1

In [29]:
processos = []

for processo in processo_json['hits']['hits']:
  num_processo = processo['_source']['numeroProcesso']
  grau = processo['_source']['grau']
  assunto = processo['_source']['assuntos']
  num_classe_processo = processo['_source']['classe']['codigo']
  classe_processo = processo['_source']['classe']['nome']
  formato_processo = processo['_source']['formato']['nome']
  tribunal = processo['_source']['tribunal']
  data_ajuizamento = processo['_source']['dataAjuizamento']
  ultima_atualizacao = processo['_source']['dataHoraUltimaAtualizacao']
  orgao_julgador = processo['_source']['orgaoJulgador']['nome']
  sort = processo['sort'][0]
  try:
    movimentos = processo['_source']['movimentos']
  except:
    movimentos = []

  processos.append([num_processo, grau, num_classe_processo, classe_processo, formato_processo,
                      tribunal, data_ajuizamento, ultima_atualizacao, orgao_julgador, assunto, movimentos, sort])


bd = pd.DataFrame(processos, columns=['Numero_processo', 'Grau_processo', 'Numero_classeProcesso', 'Classe_processo', 'Formato_processo',
                                      'Tribunal', 'DataAjuizamento', 'UltimaAtualizacao', 'Orgao_julgador', 'Assunto_processo', 'Movimentos', 'Sort'])

bd.head(1)

,Numero_processo,Grau_processo,Numero_classeProcesso,Classe_processo,Formato_processo,Tribunal,DataAjuizamento,UltimaAtualizacao,Orgao_julgador,Assunto_processo,Movimentos,Sort
0,20001169619968260103,G1,64,Ação Civil de Improbidade Administrativa,Físico,TJSP,1996-05-23T08:00:00.000Z,2024-12-12T23:53:18.355Z,VARA UNICA DE CACONDE,"[{'codigo': 10012, 'nome': 'Dano ao Erário'}]","[{'complementosTabelados': [{'codigo': 2, 'val...",832838400000


In [30]:
def converte_data(data_str):
    return pd.to_datetime(data_str).tz_convert('America/Sao_Paulo')


def gera_lista_assuntos(Assunto_processo):
  lista_assuntos = []
  for assunto in Assunto_processo:
    try:
      codigo = assunto.get('codigo')
      nome = assunto.get('nome')
      lista_assuntos.append([codigo, nome])
    except:
      lista_assuntos.append('')
  return lista_assuntos

def gera_lista_movimentos(Movimentos):
  lista_movimentos = []
  for movimento in Movimentos:
    codigo = movimento.get('codigo')
    nome = movimento.get('nome')
    data_hora = movimento.get('dataHora')

    if data_hora:
      data_hora = converte_data(data_hora)

    lista_movimentos.append([codigo, nome, data_hora])
  return lista_movimentos

In [31]:
bd['MesAjuizamento'] = pd.DatetimeIndex(bd['DataAjuizamento']).month
bd['AnoAjuizamento'] = pd.DatetimeIndex(bd['DataAjuizamento']).year

In [32]:
bd_processos = bd[['Numero_processo', 'Grau_processo', 'Numero_classeProcesso',
         "Classe_processo", "Formato_processo", "Tribunal",
         "DataAjuizamento", "AnoAjuizamento", "MesAjuizamento",
         "UltimaAtualizacao", "Orgao_julgador", "Assunto_processo",
         "Movimentos", "Sort"]]

In [33]:
bd_processos_sor = bd_processos.query('Orgao_julgador.str.contains("Sorocaba", case=False)')
bd_processos_itu = bd_processos.query('Orgao_julgador.str.contains("Itu", case=False)')

In [ ]:
entidades_processo = [] # Lista para armazenar as partes de cada processo

driver = webdriver.Chrome()
driver.get('https://esaj.tjsp.jus.br/cpopg/open.do')
driver.maximize_window()

def pesquisa(driver, nprocesso, nforo):
    try:
        botao_tipo_pesquisa = driver.find_element(By.ID, "cbPesquisa")
        select_tipo_pesquisa = Select(botao_tipo_pesquisa)
        select_tipo_pesquisa.select_by_index(0)  # Seleciona para pesquisar por NÚMERO DO PROCESSO

        campo_nprocesso = driver.find_element(By.ID, 'numeroDigitoAnoUnificado')
        campo_foronumero = driver.find_element(By.ID, 'foroNumeroUnificado')

        campo_nprocesso.clear()
        campo_foronumero.clear()

        campo_nprocesso.send_keys(nprocesso)
        campo_foronumero.send_keys(nforo)

        time.sleep(2) # Reduzi o tempo para 2 segundos para teste, ajuste conforme necessário

        search_button = driver.find_element(By.ID, "botaoConsultarProcessos") #Clica no botão para pesquisar
        search_button.click()
        time.sleep(2) # Adicionei um tempo de espera após clicar em pesquisar

    except NoSuchElementException:
        print(f"Erro ao realizar pesquisa para {nprocesso, nforo}")
        return False  # Indica falha na pesquisa

    return True

def extrai_infos(driver, entidades): # Removi 'bd' pois não está sendo usado aqui diretamente
    try:
        expande_partes = driver.find_element(By.ID, 'linkpartes')
        if expande_partes and expande_partes.is_displayed(): # Verifique se o elemento está visível antes de clicar
            expande_partes.click()
            time.sleep(2) # Adicionei um tempo de espera após expandir partes

        partes_processo = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'nomeParteEAdvogado'))
        )
        for partes in partes_processo:
            nome_parte = partes.text.strip()
            print(nome_parte)
            entidades.append(nome_parte)

    except Exception as e: # Captura Exception genérica para pegar outros possíveis erros
        partes_processo = driver.find_elements(By.CLASS_NAME, 'nomeParteEAdvogado')
        for partes in partes_processo:
            nome_parte = partes.text.strip()
            print(nome_parte)
            entidades.append(nome_parte)
    time.sleep(2) # Reduzi o tempo para 2 segundos para teste, ajuste conforme necessário


def voltar_pesquisa(driver):
    try:
        back_button = driver.find_element(By.ID, 'setaVoltar')
        back_button.click()
        time.sleep(2) # Adicionei um tempo de espera após voltar
    except NoSuchElementException:
        print("Botão 'Voltar' não encontrado ou não clicável.")

#Raspagem
count = 1
for i, row in bd_processos_sor.iterrows():
    nprocesso = (row['Numero_processo'][:13])
    nforo = (row['Numero_processo'][16:])

    print(count, "Pesquisando por", nprocesso+nforo)
    count += 1

    if pesquisa(driver, nprocesso, nforo): # Verifica se a pesquisa foi bem-sucedida
        entidades = [] # Inicializa 'entidades' *dentro* do loop para cada processo
        extrai_infos(driver, entidades) # Passa a lista 'entidades' para ser preenchida
        entidades_processo.append(entidades) # Adiciona a lista de entidades *deste* processo à lista geral
        voltar_pesquisa(driver)
    else:
        entidades_processo.append([]) # Se a pesquisa falhar, adiciona uma lista vazia para manter a correspondência com o DataFrame
        print(f"Falha na pesquisa para {nprocesso+nforo}. Partes não extraídas.")


driver.quit() # Use driver.quit() para fechar o navegador e finalizar a sessão corretamente

bd_processos['Partes'] = entidades_processo # Atribui a lista de listas 'entidades_por_processo' à coluna 'Partes'

1 Pesquisando por 00371504820060602


Ministério Público do Estado de São Paulo
Promotor:  Promotor de Justiça Dr Orlando Bastos Filho
Renato Fauvel Amary
Advogado:  Rodrigo Gomes Monteiro  
Advogada:  Claudia Cristina Ayres Amary Inomata  
Advogado:  Cid Flaquer Scartezzini  
Advogada:  Silvia Ferreira Lopes Peixoto
Pratic Service & Terceirizados
Advogada:  Caroline Oliveira Souza Mucci  
Advogado:  Carlos Cesar Pinheiro da Silva
Marcelo Forango Berenger
Carlos Roberto Levy Pinto
Advogada:  Ana Carolina Lopes  
Advogado:  Mario Jose Pustiglione Junior
Paulo Stefanius Lopes
2 Pesquisando por 00488944020060602
Ministério Público do Estado de São Paulo
Jair Ferreira Duarte Júnior
Advogada:  Iris Bardelotti Meneguetti  
Advogada:  Cintia Nuciene Sarti de Souza  
Reprtate:  Jair Ferreira Duarte Neto 
Reprtate:  Natalia Ferreira Duarte 
Reprtate:  Dario Ferreira Duarte 
Reprtate:  Idalina Maria Fonseca Ferreira Duarte
Everaldo de Moraes
Advogado:  René Ednilson da Costa Contó
3 Pesquisando 

ValueError: Length of values (118) does not match length of index (6771)

In [52]:
print(
    len(entidades_processo),
    len(bd_processos))

118 6771


In [55]:
bd_processos_sor['Partes'] = entidades_processo

C:\Users\matheus.arruda\AppData\Local\Temp\ipykernel_22260\3329724194.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bd_processos_sor['Partes'] = entidades_processo


## Ajustes

In [65]:
bd_processos_sor = bd_processos_sor.explode('Partes')

In [67]:
bd_processos_sor.drop(columns=['Movimentos'], inplace=True)

In [68]:
bd_processos_sor.to_csv('processos_impADM_nomeParte.csv', sep=',', index=False)

# Análise

In [69]:
bd_processos_sor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 966 entries, 518 to 6723
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Numero_processo        966 non-null    object
 1   Grau_processo          966 non-null    object
 2   Numero_classeProcesso  966 non-null    int64 
 3   Classe_processo        966 non-null    object
 4   Formato_processo       966 non-null    object
 5   Tribunal               966 non-null    object
 6   DataAjuizamento        966 non-null    object
 7   AnoAjuizamento         966 non-null    int32 
 8   MesAjuizamento         966 non-null    int32 
 9   UltimaAtualizacao      966 non-null    object
 10  Orgao_julgador         966 non-null    object
 11  Assunto_processo       966 non-null    object
 12  Sort                   966 non-null    int64 
 13  Partes                 965 non-null    object
dtypes: int32(2), int64(2), object(10)
memory usage: 105.7+ KB


In [70]:
bd_processos_sor.columns

Index(['Numero_processo', 'Grau_processo', 'Numero_classeProcesso',
       'Classe_processo', 'Formato_processo', 'Tribunal', 'DataAjuizamento',
       'AnoAjuizamento', 'MesAjuizamento', 'UltimaAtualizacao',
       'Orgao_julgador', 'Assunto_processo', 'Sort', 'Partes'],
      dtype='object')

In [71]:
bd_processos_sor[['Numero_processo', 'AnoAjuizamento', 'Assunto_processo','Partes']]

,Numero_processo,AnoAjuizamento,Assunto_processo,Partes
518,00371504820068260602,2006,"[{'codigo': 899, 'nome': 'DIREITO CIVIL'}]",
518,00371504820068260602,2006,"[{'codigo': 899, 'nome': 'DIREITO CIVIL'}]",
518,00371504820068260602,2006,"[{'codigo': 899, 'nome': 'DIREITO CIVIL'}]",Ministério Público do Estado de São Paulo\nPro...
518,00371504820068260602,2006,"[{'codigo': 899, 'nome': 'DIREITO CIVIL'}]",Renato Fauvel Amary\nAdvogado: Rodrigo Gomes ...
518,00371504820068260602,2006,"[{'codigo': 899, 'nome': 'DIREITO CIVIL'}]",Pratic Service & Terceirizados\nAdvogada: Car...
...,...,...,...,...
6723,10381737920248260602,2024,"[{'codigo': 10014, 'nome': 'Violação dos Princ...",Ministério Público do Estado de São Paulo
6723,10381737920248260602,2024,"[{'codigo': 10014, 'nome': 'Violação dos Princ...",Rodrigo Maganhoto
6723,10381737920248260602,2024,"[{'codigo': 10014, 'nome': 'Violação dos Princ...",Cleber Martins Fernandes Costa
6723,10381737920248260602,2024,"[{'codigo': 10014, 'nome': 'Violação dos Princ...",Marcelo Duarte Regalado


In [99]:
bd_processos_sor['Partes'].value_counts()

Partes
                                                                                                                      217
Ministério Público do Estado de São Paulo                                                                              32
PREFEITURA MUNICIPAL DE SOROCABA                                                                                       30
Fazenda Pública do Estado de São Paulo                                                                                 18
Ministerio Publico                                                                                                      9
                                                                                                                     ... 
Januário Renna\nAdvogado:  Rodrigo Benedito Tarossi                                                                     1
PREFEITURA MUNICIPAL DE SOROCABA\nAdvogado:  Vilton Luis da Silva Barboza  \nAdvogado:  Douglas Domingos de Moraes      1
Claudemir José Ju

In [98]:
bd_processos_sor.query('Partes == "Lippi"')

,Numero_processo,Grau_processo,Numero_classeProcesso,Classe_processo,Formato_processo,Tribunal,DataAjuizamento,AnoAjuizamento,MesAjuizamento,UltimaAtualizacao,Orgao_julgador,Assunto_processo,Sort,Partes


In [97]:
bd_processos_sor.query('Partes.str.contains("Rodrigo")')

ValueError: unknown type object

In [91]:
bd_processos_sor.query('Partes.str.contains("Rodrigo", case=False)')

ValueError: unknown type object

In [89]:
bd_processos_sor.query('Partes.str.contains("Lippi", case=False)')['Numero_processo'].unique()

ValueError: unknown type object